# EDA de **Telefonía móvil: Penetración (Accesos cada 100 habitantes)**

Notebook preparado para **Visual Studio/VS Code** con Python, con explicaciones en Markdown entre pasos y guardado del CSV 'limpio'.

**Fuente de datos (ruta Windows):**

`C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Comunicaciones  Moviles\\Telefonía móvil Penetración_con_ID.csv`


## 1. Importaciones y configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (11, 5),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
Se preserva la codificación **UTF‑8 con BOM** para acentos/eñes en Windows.

In [ ]:
ruta = r'C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Comunicaciones  Moviles\\Telefonía móvil Penetración_con_ID.csv'

# Cargar el archivo
df = pd.read_csv(ruta, encoding='utf-8-sig')

# Copia del crudo
df_raw = df.copy()

print('Shape:', df.shape)
print('Columnas:', list(df.columns))
display(df.head())

## 3. Perfil inicial del dataset
- Tipos de datos y memoria (`df.info()`).
- Valores faltantes y duplicados.
- Estadísticos descriptivos de `Accesos cada 100 hab`.

In [ ]:
df.info()
print('
Nulos por columna:
', df.isna().sum())
print('Duplicados (filas completas):', df.duplicated().sum())

if 'Accesos cada 100 hab' in df.columns:
    print('
Estadísticos Accesos cada 100 hab:')
    print(df['Accesos cada 100 hab'].describe())

## 4. Validaciones de consistencia y plausibilidad
- Conversión a numérico.
- Chequeo de negativos.
- Detección de valores fuera de rango esperado (por ejemplo 0–300).
- Construcción de `Periodo` (YYYY-Qn).

In [ ]:
num_cols = ['Registro_ID','Año','Trimestre','Accesos cada 100 hab','TiempoTrim_ID']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('
Negativos por columna:
', (df[num_cols] < 0).sum())

# Valores fuera de rango (0-300) para penetración
low, high = 0, 300
out_rango = df[(df['Accesos cada 100 hab'] < low) | (df['Accesos cada 100 hab'] > high)]
print(f'Fuera de rango (0-300): {len(out_rango)} registros')

# Periodo YYYY-Qn
df['Periodo'] = pd.PeriodIndex.from_fields(year=df['Año'], quarter=df['Trimestre']).astype(str)

## 5. Limpieza y ordenamiento
- Eliminación de duplicados.
- Orden temporal por `Año` y `Trimestre`.

In [ ]:
df = df.drop_duplicates().sort_values(['Año','Trimestre']).reset_index(drop=True)
display(df.head())

## 6. Variables derivadas
- `Pen_QoQ_%` (variación trimestral).
- `Pen_YoY_%` (variación interanual t-4).
- `Pen_MovAvg4` (media móvil de 4 trimestres).

In [ ]:
df['Pen_QoQ_%'] = df['Accesos cada 100 hab'].pct_change() * 100
df['Pen_YoY_%'] = df['Accesos cada 100 hab'].pct_change(4) * 100
df['Pen_MovAvg4'] = df['Accesos cada 100 hab'].rolling(4).mean()

display(df[['Año','Trimestre','Accesos cada 100 hab','Pen_QoQ_%','Pen_YoY_%','Pen_MovAvg4']].tail(8))

## 7. Visualizaciones
- Serie temporal de **Accesos cada 100 hab** y su media móvil (4).
- Variación `Pen_YoY_%` y `Pen_QoQ_%`.

In [ ]:
import matplotlib.ticker as mtick

# 7.1 Penetración y media móvil
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Accesos cada 100 hab', marker='o', label='Accesos/100 hab', color='#2E86C1')
sns.lineplot(data=df, x='Periodo', y='Pen_MovAvg4', marker='o', label='Media móvil (4)', color='#7D3C98')
plt.xticks(rotation=45, ha='right')
plt.title('Penetración móvil (Accesos cada 100 habitantes)')
plt.xlabel('Periodo'); plt.ylabel('Accesos/100 hab'); plt.legend(); plt.tight_layout(); plt.show()

# 7.2 Variación YoY y QoQ
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Pen_YoY_%', marker='o', label='YoY %', color='#1ABC9C')
sns.lineplot(data=df, x='Periodo', y='Pen_QoQ_%', marker='o', label='QoQ %', color='#E67E22')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.xticks(rotation=45, ha='right')
plt.title('Variación de Penetración (YoY y QoQ)')
plt.xlabel('Periodo'); plt.ylabel('%'); plt.legend(); plt.tight_layout(); plt.show()

## 8. Guardado del archivo **limpio**
Se guarda el CSV en la **misma ruta** con la **leyenda `_limpio`** agregada al nombre, codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)